In [1]:
import gdspy
import numpy as np


In [2]:
# The GDSII file is called a library, which contains multiple cells.
lib = gdspy.GdsLibrary(unit=1e-6, precision=1e-9)

# Geometry must be placed in cells.
cell = lib.new_cell('IDT_waveguide')

tol = 0.0001 # This control how good circles/arcs really is
fontsize = 7


In [3]:
# IDT inputs
pitch = [0.12*4]
dutycycle = 0.5
finger_len = 9.0 # needs to be smaller than 'd_in'
N_finger_pair = [5, 10, 20]
fillet_finger = 1
rotations_geometry = -np.array([90, 0])*np.pi/180
rotations_material = -rotations_geometry

# IDT/PAD region
overlap_pad_width = 2.0
overlap_pad_X = 29.5
overlap_pad_Y = 5.0
fillet_pad = 1
fillet_overlap = 1
d_in = 10.0
d_out = [5.0, 20.0, 50.0]
pad_size = [75.0, 100.0]
#-------------

# Wafer/E-Beam Input
sample_number = "CCR_IDT_003"
ebeam_brand = "EL7"
ebeam_energy = 150 # in keV
ebeam_finger_dose = "1x1000=1000" # in uC/cm2
ebeam_finger_dose_incremental = "-100"
ebeam_pad_dose = "1x1200=1200" # in uC/cm2
ebeam_finger_current = 0.2 # in nA
ebeam_pad_current = 20 # in nA
resist = "PMMA"
field_size = 500

LN_thickness = 0.6
cut = "X"
rotate_axes = 90*(np.pi/180)
#-------------


In [4]:
finger_layer=0
pad_layer=5
header_layer=200
big_header_layer = 201

for m in range(len(d_out)):
    dx_m = 0
    dy_m = -m*650
    
    for k in range(len(rotations_geometry)):
        xr = 148.883 # x center of rotation
        yr = -275.284 # y center of rotation
        dx_k = k*700
        dy_k = 0
        y0 = 0
        for i in range(len(N_finger_pair)):
            for j in range(len(pitch)):
                dx = dx_m + dx_k
                dy = dy_m + dy_k
                x0 = j*2*(d_in/2 + 3*overlap_pad_width/2 + overlap_pad_X + pad_size[0]/2)
                if i!=0 and j==0:
                    y1 = 2*pad_size[1] + d_out[m] + 2*overlap_pad_Y + 2*(N_finger_pair[i-1] + 1/4)*np.max(pitch) + overlap_pad_width
                    y2 = 2*pad_size[1] + d_out[m] + 2*overlap_pad_Y + 2*(N_finger_pair[i] + 1/4)*np.max(pitch) + overlap_pad_width
                    y0 = y0 - (y2+y1)/2 + pad_size[1]

                # Finger Geometry with pitch = pitch[j]
                finger_points = [
                    (-finger_len/2 - overlap_pad_width/4, 0),
                    (-finger_len/2 - overlap_pad_width/4, pitch[j]*dutycycle/2),
                    (finger_len/2 + overlap_pad_width/4, pitch[j]*dutycycle/2),
                    (finger_len/2 + overlap_pad_width/4, 0)
                ]
                #----------
                for mirror in [0,1]:
                    for pair_idx in range(N_finger_pair[i]):
                        # Right Finger position
                        finger_right = gdspy.Polygon(finger_points, layer=finger_layer)
                        finger_right.translate(d_in/2 - finger_len/2 + overlap_pad_width/4, (pair_idx + dutycycle/2)*pitch[j])

                        # Left Finger position
                        finger_left = gdspy.Polygon(finger_points, layer=finger_layer)
                        finger_left.translate(-d_in/2 + finger_len/2 - overlap_pad_width/4, (pair_idx + dutycycle/2 + 1/2)*pitch[j])

                        # Fillet
                        if fillet_finger:
                            finger_right.fillet(1)
                            finger_left.fillet(1)

                        # Bottom part of the IDT as a mirror of the top part
                        if mirror:
                            finger_right.mirror((1,0), (0,0))
                            finger_right.translate(0, -d_out[m])

                            finger_left.mirror((1,0), (0,0))
                            finger_left.translate(0, -d_out[m])

                        finger_right.translate(x0, y0)
                        finger_left.translate(x0, y0)

                        finger_right.rotate(rotations_geometry[k], (xr, yr))
                        finger_left.rotate(rotations_geometry[k], (xr, yr))

                        finger_right.translate(dx, dy)
                        finger_left.translate(dx, dy)

                        # Add fingers for each 'pair_idx'
                        cell.add(finger_right)
                        cell.add(finger_left)

                    # Finger/Overlap region
                    extra_finger_overlap = 2

                    finger_overlap_left = gdspy.Path(overlap_pad_width/2, (-overlap_pad_width/2-d_in/2, 0))
                    finger_overlap_left.segment((N_finger_pair[i] + 1/4)*pitch[j] + extra_finger_overlap, '+y', layer=finger_layer)
                        
                    finger_overlap_right = gdspy.Path(overlap_pad_width/2, (overlap_pad_width/2+d_in/2, 0))
                    finger_overlap_right.segment((N_finger_pair[i] + 1/4)*pitch[j] + extra_finger_overlap, '+y', layer=finger_layer)
                        
                    # Overlap/PAD region
                    extra_pad_overlap = 10
                    
                    pad_overlap_left = gdspy.Path(overlap_pad_width, (-overlap_pad_width/2-d_in/2, (N_finger_pair[i] + 1/4)*pitch[j]))
                    pad_overlap_left.segment(overlap_pad_Y, '+y', layer=pad_layer)
                    pad_overlap_left.arc(overlap_pad_width, 0, np.pi/2, layer=pad_layer)
                    pad_overlap_left.segment(overlap_pad_X + extra_pad_overlap, layer=pad_layer)

                    pad_overlap_right = gdspy.Path(overlap_pad_width, (overlap_pad_width/2+d_in/2, (N_finger_pair[i] + 1/4)*pitch[j]))
                    pad_overlap_right.segment(overlap_pad_Y, '+y', layer=pad_layer)
                    pad_overlap_right.arc(overlap_pad_width, np.pi, np.pi/2, layer=pad_layer)
                    pad_overlap_right.segment(overlap_pad_X + extra_pad_overlap, layer=pad_layer)
                    #----------

                    # PADs
                    pad_left = gdspy.Rectangle((-pad_size[0], 0), (0, pad_size[1]), layer=pad_layer)
                    pad_left.translate(-d_in/2 - 3*overlap_pad_width/2 - overlap_pad_X, (N_finger_pair[i] + 1/4)*pitch[j] + overlap_pad_width/2 + overlap_pad_Y)

                    pad_right = gdspy.Rectangle((0, 0), (pad_size[0], pad_size[1]), layer=pad_layer)
                    pad_right.translate(d_in/2 + 3*overlap_pad_width/2 + overlap_pad_X, (N_finger_pair[i] + 1/4)*pitch[j] + overlap_pad_width/2 + overlap_pad_Y)

                    if fillet_pad:
                        pad_left.fillet(1)
                        pad_right.fillet(1)
                    
                    if mirror: 
                        finger_overlap_right.mirror((1,0), (0,0))
                        finger_overlap_left.mirror((1,0), (0,0))
                        pad_overlap_left.mirror((1,0), (0,0))
                        pad_overlap_right.mirror((1,0), (0,0))
                        pad_left.mirror((1,0), (0,0))
                        pad_right.mirror((1,0), (0,0))

                        finger_overlap_right.translate(0, -d_out[m])
                        finger_overlap_left.translate(0, -d_out[m])
                        pad_overlap_left.translate(0, -d_out[m])
                        pad_overlap_right.translate(0, -d_out[m])
                        pad_left.translate(0, -d_out[m])
                        pad_right.translate(0, -d_out[m])

                    finger_overlap_right.translate(x0, y0)
                    finger_overlap_left.translate(x0, y0)
                    pad_overlap_left.translate(x0, y0)
                    pad_overlap_right.translate(x0, y0)
                    pad_left.translate(x0, y0)
                    pad_right.translate(x0, y0)

                    finger_overlap_right.rotate(rotations_geometry[k], (xr, yr))
                    finger_overlap_left.rotate(rotations_geometry[k], (xr, yr))
                    pad_overlap_left.rotate(rotations_geometry[k], (xr, yr))
                    pad_overlap_right.rotate(rotations_geometry[k], (xr, yr))
                    pad_left.rotate(rotations_geometry[k], (xr, yr))
                    pad_right.rotate(rotations_geometry[k], (xr, yr))  

                    finger_overlap_right.translate(dx, dy)
                    finger_overlap_left.translate(dx, dy)
                    pad_overlap_left.translate(dx, dy)
                    pad_overlap_right.translate(dx, dy)
                    pad_left.translate(dx, dy)
                    pad_right.translate(dx, dy)
                    
                    if fillet_overlap:
                        pad_overlap_left_fillet = gdspy.boolean(pad_overlap_left, None, "or", max_points=0, layer=pad_layer)
                        pad_overlap_left_fillet.fillet(1)
                        
                        pad_overlap_right_fillet = gdspy.boolean(pad_overlap_right, None, "or", max_points=0, layer=pad_layer)
                        pad_overlap_right_fillet.fillet(1)
                        
                        cell.add(pad_overlap_left_fillet)
                        cell.add(pad_overlap_right_fillet)
                    else:
                        cell.add(pad_overlap_left)
                        cell.add(pad_overlap_right)
                    
                    cell.add(finger_overlap_right)
                    cell.add(finger_overlap_left)
                    cell.add(pad_left)
                    cell.add(pad_right)
                    #----------
                
                header1 = "2N:"+str(int(N_finger_pair[i]))+"\n"
                header2 = "p:"+str(round(pitch[j], 3))+"\n"
                header3 = "A:"+str(round(rotations_material[k]*180/np.pi, 3))+"\n"
                header4 = "d:"+str(round(d_out[m], 3))+"\n"
                header_all = header1+header2+header3+header4
                header = gdspy.Text(header_all, fontsize, layer=header_layer)
                header.translate(x0, y0)
                dx_header = -17
                dy_header = (N_finger_pair[i] + 1/4)*pitch[j] + 62
                header.translate(dx_header, dy_header)
                header.rotate(rotations_geometry[k], (xr, yr))
                header.translate(dx, dy)
                cell.add(header)
                
header1 = sample_number+"\n\n"
header2 = "Brand:"+ebeam_brand+"\n"
header3 = "Energy:"+str(ebeam_energy)+"keV \n"
header4 = "Writting Field:"+str(field_size)+"um \n\n"
header5 = "Finger Current:"+str(ebeam_finger_current)+"nA \n"
header6 = "Finger Dose:"+str(ebeam_finger_dose)+"uC/cm2 \n"
header7 = "Finger Dose Incremental:"+str(ebeam_finger_dose_incremental)+"uC/cm2 \n\n"
header8 = "PAD Current:"+str(ebeam_pad_current)+"nA \n"
header9 = "PAD Dose:"+str(ebeam_pad_dose)+"uC/cm2 \n\n"
header10 = "Resist:"+str(resist)+" \n\n"
header11 = "t:"+str(round(LN_thickness, 3))+"\n"
header12 = "duty:"+str(round(dutycycle, 3))+"\n"
header13 = "L:"+str(round(finger_len, 3))+"\n"
header14 = "d_in:"+str(round(d_in, 3))+"\n"
header_all = header1+header2+header3+header4+header5+header6+header7+header8+header9+header10+header11+header12+header13+header14
header = gdspy.Text(header_all, 10, layer=big_header_layer)
dx_header = -110
dy_header = 550
header.translate(dx_header, dy_header)
cell.add(header)

if cut == "Z" or cut == "X": 
    # axes
    dx_axes = 40
    dy_axes = 300
    material_axis = gdspy.Path(3, (dx_axes, dy_axes))
    material_axis.segment(50, "-y", layer=big_header_layer)
    material_axis.arc(3, -np.pi, -np.pi/2, layer=big_header_layer)
    material_axis.segment(50, "+x", layer=big_header_layer)
    material_axis.rotate(rotate_axes, (dx_axes, dy_axes))
    
    if cut == "Z":
        material_X_label = gdspy.Text("X", 10, (dx_axes+50, dy_axes-50), layer=big_header_layer)
        material_X_label.rotate(rotate_axes, (dx_axes+50, dy_axes-50))
        
        material_Y_label = gdspy.Text("Y", 10, (dx_axes, dy_axes), layer=big_header_layer)
        material_Y_label.rotate(rotate_axes, (dx_axes, dy_axes))
        
        brand = gdspy.Text("Z-cut", 10, (dx_axes+50, dy_axes), layer=big_header_layer)
        brand.rotate(rotate_axes, (dx_axes+50, dy_axes))
        
    if cut == "X":
        material_X_label = gdspy.Text("Y", 10, (dx_axes+50, dy_axes-50), layer=big_header_layer)
        material_X_label.rotate(rotate_axes, (dx_axes+50*0, dy_axes-50*0))
        
        material_Y_label = gdspy.Text("Z", 10, (dx_axes, dy_axes), layer=big_header_layer)
        material_Y_label.rotate(rotate_axes, (dx_axes, dy_axes))
        
        brand = gdspy.Text("X-cut", 10, (dx_axes+50, dy_axes), layer=big_header_layer)
        brand.rotate(rotate_axes, (dx_axes+50*0, dy_axes))

    cell.add(material_axis)
    cell.add(material_X_label)
    cell.add(material_Y_label)
    cell.add(brand)
                            

In [5]:
# for i in np.arange(10):
#     for j in np.arange(5):
#         write_field = gdspy.Rectangle((0, 0), (field_size, field_size), layer=255)
#         write_field.translate(i*field_size-350+164, j*field_size-1900)
#         cell.add(write_field)
        

In [6]:
# Save the library in a file called 'first.gds'.
lib.write_gds(sample_number+'.gds')
